<a href="https://colab.research.google.com/github/timurgen/collabs/blob/main/v2_Introduction_to_Cognite_Python_SDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Follow the Section Links to read the Cognite Learn content before running code examples.

##[1. Environment Set Up](https://cognite.talentlms.com/unit/view/id:2116)

###Install the Cognite SDK package

If you recieve the errors:

`ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.`

`ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.`

You can disregard them and do not need to click "Restart Runtime".

In [ ]:
!pip install "cognite-sdk>=1.1.10"
!pip install --upgrade numpy

###Import other required packages

In [ ]:
%matplotlib inline

import os
from datetime import datetime, timedelta
from datetime import datetime
from getpass import getpass

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from cognite.client import CogniteClient

### Connect to Cognite Data Fusion
This client object is how all queries will be sent to the Cognite API to retrieve data.

When prompted for your API key, use the stored key generated previously in the course.

In [ ]:
client = CogniteClient(api_key=getpass("Open Industrial Data API-KEY: "),
                       project="publicdata", client_name="OID_example")

##[2. Retrieving Lists of Assets](https://cognite.talentlms.com/unit/view/id:2118)

###List assets
The `client.assets.list(limit=20)` function retrieves the first `limit` assets, and returns it as an `AssetList`.

In [ ]:
client.assets.list(limit=20)

##Search Assets##
The `client.assets.search()` function allows you to search by a specific property of the asset, including its name, parent, etc.

###Fuzzy Search by name
The search by name includes results that are similar in name, but not an exact match.

In [ ]:
asset_name = "23-HA-9103"
assets = client.assets.search(name=asset_name)
assets[:5]

###Specific Search
The `client.assets.retrieve()` interface provides the same information for one specific asset based on the provided ID or external ID.

In [ ]:
asset_id = [a.id for a in assets if a.name==asset_name][0]
client.assets.retrieve(id=asset_id)

##[3. Asset Hierarchy and Relationships](https://cognite.talentlms.com/unit/view/id:2124)


We will generate a list of all children of the main asset of interest. The main asset of interest is listed first, then the children are listed underneath in following rows.

In [ ]:
subtree = client.assets.retrieve_subtree(id=asset_id)
subtree[:5]

##[4. Collecting Time Series Data](https://cognite.talentlms.com/unit/view/id:2119)

###Compile a list of time series objects under the asset
For each of the assets in the subtree we retrieved, we get the associated time series objects and merge them into a single `TimeSeriesList` object.

In [ ]:
all_timeseries = subtree.time_series()
print(len(all_timeseries),'time series in subtree')
all_timeseries[:5]

If you are curious about which asset a time series is attached to, you can retrieve more information of the asset by using the retrieve function. Note that the property is called `asset_id` following typical python style, while `assetId` is used in the underlying API objects and tabular outputs.

In [ ]:
client.assets.retrieve(id=all_timeseries[0].asset_id)

###View datapoints for specific time series
The identifier to retrieve Datapoints is the externalId column from the output above.

In [ ]:
client.datapoints.retrieve(external_id="pi:160184", start="10d-ago", end="now")[:10]

##[5. Use Cases of CDF Data](https://cognite.talentlms.com/unit/view/id:2120)

###Collect datapoints from CDF
The time series names are defined in the in_ts_exids and out_ts_exid lists below.

In [ ]:
in_ts_exids = ["pi:160182", "pi:160697", "pi:160882"]
out_ts_exid = "pi:160696"

###Retrieve Data Points from CDF
Most object types in the Python SDK have a `to_pandas` method which converts the result to a pandas dataframe. For retrieving aggregates such as the average over each time period, you can use `client.datapoints.retrieve_dataframe` to get a pandas dataframe directly. 

In [ ]:
ts_exids = in_ts_exids + [out_ts_exid]

train_start_date = datetime(2018, 8, 1)

train_end_date = train_start_date + timedelta(days=30)

datapoints_df = client.datapoints.retrieve_dataframe(external_id=ts_exids,
                                                     aggregates=['average'],
                                                     granularity='1m',
                                                     start=train_start_date,
                                                     end=train_end_date,
                                                     include_aggregate_name=False
                                                     )
datapoints_df.fillna(method="ffill", inplace=True)
datapoints_df.head()

There are also shortcuts for filling the dataframe when using interpolation or count aggregates. Note that without the `include_aggregate_name=False` option, the aggregate name is appended to the external id to form a unique column name.

In [ ]:
datapoints_df_interp = client.datapoints.retrieve_dataframe(external_id=ts_exids[0:2],
                                                           aggregates=['interpolation','count'],
                                                           granularity='1h',
                                                           start=train_start_date,
                                                           end=train_end_date,
                                                           complete="fill"
                                                          )
datapoints_df_interp.head()

###Visualize the Time Series Data
The bottom right plot is the output time series, while the other 3 are the inputs used to create an estimate for the output.

In [ ]:
cols = datapoints_df.columns

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
for i, col in enumerate(cols):
    datapoints_df.loc[:, [col]].plot(ax=axes.ravel()[i])

##[6. Model Creation](https://cognite.talentlms.com/unit/view/id:2121)

###Gathering Training Data

In [ ]:
train_X = datapoints_df[in_ts_exids].to_numpy()
train_y = datapoints_df[out_ts_exid].to_numpy()

###Get a separate DataFrame from CDF
The data which we will use to predict the output pressure will be stored in a seperate dataframe as collected below.

In [ ]:
predict_start_date = train_end_date
# Make the prediction on 1 hour of data
predict_end_date = train_end_date + timedelta(hours=1)
predict_df = client.datapoints.retrieve_dataframe(
    external_id=ts_exids,
    aggregates=['average'],
    granularity='1m',
    start=predict_start_date,
    end=predict_end_date,
    include_aggregate_name=False
)
predict_df.fillna(method="ffill", inplace = True)
predict_df.head()

In [ ]:
cols = predict_df.columns

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
for i, col in enumerate(cols):
    predict_df.plot(y=col, ax=axes.ravel()[i]);

##[7. Linear Regression Model](https://cognite.talentlms.com/unit/view/id:2122)
As a simple starting point we will check to see how a linear regression model performs to predict the output pressure.

###Utilize sklearn to create a basic linear regression model
Sklearn is common package utilized to import and deploy data science models. Linear Regression is only one of many options for constructing models.

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(train_X, train_y)

X = predict_df[in_ts_exids].values
predict_df["prediction_lin_reg1"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(predict_df[out_ts_exid], predict_df["prediction_lin_reg1"])
r2_s = r2_score(predict_df[out_ts_exid], predict_df["prediction_lin_reg1"])
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 4)))
print('The R2 score of our forecasts is {}'.format(round(r2_s, 4)))

predict_df.plot(y=[out_ts_exid, "prediction_lin_reg1"], figsize=(10,10));

###Look at the fit for the training data

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(train_X, train_y)

X = datapoints_df[in_ts_exids].values
datapoints_df["prediction_lin_reg1"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(predict_df[out_ts_exid], predict_df["prediction_lin_reg1"])
r2_s = r2_score(predict_df[out_ts_exid], predict_df["prediction_lin_reg1"])
print('The Mean Squared Error on the training data is {}'.format(round(mse, 4)))
print('The R2 score of our training data is {}'.format(round(r2_s, 4)))

datapoints_df.plot(y=[out_ts_exid, "prediction_lin_reg1"], figsize=(10,10));

###Add dummy variable for anomalous period

In [ ]:
datapoints_df['state'] = (datapoints_df[out_ts_exid]< 4)*1
predict_df['state'] = (predict_df[out_ts_exid]< 4)*1

In [ ]:
train_X2 = datapoints_df[in_ts_exids + ['state']].values

lin_reg = LinearRegression()
lin_reg.fit(train_X2, train_y)
X = predict_df[in_ts_exids + ['state']].values
predict_df["prediction_lin_reg2"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(predict_df[out_ts_exid], predict_df["prediction_lin_reg2"])
r2_s = r2_score(predict_df[out_ts_exid], predict_df["prediction_lin_reg2"])
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 4)))
print('The R2 score of our forecasts is {}'.format(round(r2_s, 4)))

predict_df.plot(y=[out_ts_exid, "prediction_lin_reg2"], figsize=(10,10));

###Look at the fit for the training data

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(train_X2, train_y)

X = datapoints_df[in_ts_exids + ['state']].values
datapoints_df["prediction_lin_reg2"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(datapoints_df[out_ts_exid], datapoints_df["prediction_lin_reg2"])
r2_s = r2_score(datapoints_df[out_ts_exid], datapoints_df["prediction_lin_reg2"])
print('The Mean Squared Error on the training data is {}'.format(round(mse, 4)))
print('The R2 score of our training data is {}'.format(round(r2_s, 4)))

datapoints_df.plot(y=[out_ts_exid, "prediction_lin_reg2"], figsize=(10,10));

###Remove Outliers

In [ ]:
quantiles = [0.95, 0.975, 0.98, 0.99]
quantiles_df = pd.DataFrame(
    {
        "quantile": quantiles,
        "value": np.quantile(datapoints_df[out_ts_exid], q=quantiles),
    }
)
quantiles_df

In [ ]:
datapoints_df_adj = datapoints_df.loc[datapoints_df[out_ts_exid] < 4, :]


In [ ]:
train_X_adj = datapoints_df_adj[in_ts_exids].values
train_y_adj = datapoints_df_adj[out_ts_exid].values

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(train_X_adj, train_y_adj)

X = predict_df[in_ts_exids].values
predict_df["prediction_lin_reg3"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(predict_df[out_ts_exid], predict_df["prediction_lin_reg3"])
r2_s = r2_score(predict_df[out_ts_exid], predict_df["prediction_lin_reg3"])
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 6)))
print('The R2 score of our forecasts is {}'.format(round(r2_s, 6)))

predict_df.plot(y=[out_ts_exid, "prediction_lin_reg3"], figsize=(10,10));

###Look at the fit for the training data

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(train_X_adj, train_y_adj)

X = datapoints_df[in_ts_exids].values
datapoints_df["prediction_lin_reg3"] = lin_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(datapoints_df[out_ts_exid], datapoints_df["prediction_lin_reg3"])
r2_s = r2_score(datapoints_df[out_ts_exid], datapoints_df["prediction_lin_reg3"])
print('The Mean Squared Error on the training data is {}'.format(round(mse, 4)))
print('The R2 score of our training data is {}'.format(round(r2_s, 4)))

datapoints_df.plot(y=[out_ts_exid, "prediction_lin_reg3"], figsize=(10,10));

##[8. Random Forest Ensemble Model](https://cognite.talentlms.com/unit/view/id:2123)

In [ ]:
rnd_forest_reg = RandomForestRegressor(n_estimators=10, min_samples_split=20, max_depth=5)
rnd_forest_reg.fit(train_X, train_y)

X = predict_df[in_ts_exids].values
predict_df["prediction_rnd_forest"] = rnd_forest_reg.predict(X)

# print out mse of the prediction
mse = mean_squared_error(predict_df[out_ts_exid], predict_df["prediction_rnd_forest"])
r2_s = r2_score(predict_df[out_ts_exid], predict_df["prediction_rnd_forest"])
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 4)))
print('The R2 score of our forecasts is {}'.format(round(r2_s, 4)))

predict_df.plot(y=[out_ts_exid, "prediction_rnd_forest"], figsize=(10,10));

In [ ]:
rnd_forest_reg = RandomForestRegressor(n_estimators=10, min_samples_split=20, max_depth=5)
rnd_forest_reg.fit(train_X, train_y)

X = datapoints_df[in_ts_exids].values
datapoints_df["prediction_rnd_forest"] = rnd_forest_reg.predict(X)

datapoints_df.plot(y=[out_ts_exid, "prediction_rnd_forest"], figsize=(10,10));

###Anomaly Detection

In [ ]:
#Train up until 100 timestamps before anomalous period
datapoints_df = datapoints_df.assign(
    datetime=datapoints_df.index
).reset_index(drop=True)

predict_start_index = min(datapoints_df[datapoints_df[out_ts_exid] > 5].index) - 100

datapoints_df_ad = datapoints_df.loc[:predict_start_index, :]
train_X = datapoints_df_ad[in_ts_exids].values
train_y = datapoints_df_ad[out_ts_exid].values

predict_df_ad = datapoints_df.loc[predict_start_index+1:, in_ts_exids + [out_ts_exid, "datetime"]]

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(datapoints_df_ad["datetime"], datapoints_df_ad[out_ts_exid], label="train")
plt.plot(predict_df_ad["datetime"], predict_df_ad[out_ts_exid], label="predict")
plt.legend()
plt.xlabel("datetime")
plt.title(out_ts_exid)

In [ ]:
rnd_forest_reg = RandomForestRegressor(n_estimators=10, min_samples_split=20, max_depth=5)
rnd_forest_reg.fit(train_X, train_y)

X = predict_df_ad[in_ts_exids].values
predict_df_ad["prediction_rnd_forest"] = rnd_forest_reg.predict(X)
predict_df_ad["residual"] = np.abs(predict_df_ad["prediction_rnd_forest"]-predict_df_ad[out_ts_exid])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,15))
predict_df_ad.plot(
    x="datetime",
    y=[out_ts_exid, "prediction_rnd_forest"],
    figsize=(12,7),
    ax=ax1, 
    color=["C1", "C2"],
);
predict_df_ad.plot(
    x="datetime",
    y=["residual"],
    figsize=(12,7),
    ax=ax2,
    color="C3",
);